<a href="https://colab.research.google.com/github/IPMSand/NLP-Module/blob/main/Sentiment_Analysis(2)ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis

In [5]:
import re

In [1]:
# Lexciob:word -> sentiment score(positive > 0, negative <0)
lexicon = {
    'good': 1.0,
    'bad': -1.5,
    'terrible': -1.5,
    'love': 2.0,
    'hate': -2.0,
    'happy': 1.5,
    'sad': -1.5,

}


In [2]:
# Negation Words
negations =   {'not','no', 'never','none'}

In [6]:
def preprocess_text(text):
  """
  Preprocesses text by converting to lowercase and tokenizing into words.
  Uses re.findall as simple tokenizer to extract words.
  """
  text = text.lower()
  #
  words = re.findall(r'\b\w+\b', text)
  return words


def lexicon_sentiment(text):
  """
  Performs lexicon-basedd sentiment analysis with negation handling.
  Returns sentiment based on summed word scores.
  """
  words = preprocess_text(text)
  score = 0.0
  negate = False
  for word in words:
    if word in negations:
      negate = True # set flag to negate next sentiment word
      continue
    if word in lexicon:
      word_score = lexicon[word]
      if negate: # if negate is True
        word_score = -word_score # flip the polarity of the score
        negate = False
      score += word_score
  if score > 0:
    return 'positive'
  elif score < 0:
    return 'negative'
  else:
    return 'neutral'

In [12]:
# Example Usage
text = "I love this product, it's not bad at all!"
print(lexicon_sentiment(text)) # output : Positive

positive


In [13]:
# Example Usage 2
text = "I love this product, it's not bad at all!. I thought I will hate it really badly when I saw it's teribble state, Hate it!!"
print(lexicon_sentiment(text)) # Output

negative
